In [ ]:
!pip install opencv-python

In [4]:
from keras.models import load_model
from time import sleep
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np

In [5]:
face_classifier = cv2.CascadeClassifier("./haarcascade_frontalface_default.xml")
classifier = load_model("./MY_DCNN_1.h5")

#,'Happy','Neutral', 'Sad', 'Surprise'
# for MoMoModel
emotion_labels = ['Angry','Disgust','Fear','Happy','Sad', 'Suprise', 'Neutral']

# for DCNN.h5
#emotion_labels = ['anger','fear','happy','sad','neutral']

#for DCNN_2 for LSTM IT DOES NOT WORK PROPERLY
#emotion_labels = ['Surpreso','Feliz','Bravo','Triste','Medo']

cap = cv2.VideoCapture(0)

In [ ]:
while True:
    _, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray)

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,255),2)
        roi_gray = gray[y:y+h,x:x+w]
        roi_gray = cv2.resize(roi_gray,(48,48),interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray])!=0:
            roi = roi_gray.astype('float')/255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi,axis=0)

            prediction = classifier.predict(roi)[0]
            #print("\nprediction = ",prediction)
            label = emotion_labels[prediction.argmax()]
            #print("\nprediction max = ",label)
            label_position = (x,y-10)
            cv2.putText(frame,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
        else:
            cv2.putText(frame,'No Faces',(30,80),cv2.FONT_HERSHEY_SIMPLEX,1,(0,255,0),2)
    cv2.imshow('Emotion Detector',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()